In [1]:
import pandas as pd
import numpy as np

import codecs, json

from collections import Counter

In [2]:
# Initialize empty df for the 106 brands
index = np.arange(106)
columns = ['brand_id', 'brand_name', 'num_tot_posts', 'num_en_posts', 
           'num_it_posts', 'num_mturk', 'num_tribe', 'model_decision_empty']

df_res = pd.DataFrame(index=index, columns=columns)
df_res = df_res.fillna(0) # with 0s rather than NaNs

df_res.head(3)

,brand_id,brand_name,num_tot_posts,num_en_posts,num_it_posts,num_mturk,num_tribe,model_decision_empty
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0


In [3]:
# Initalize empty lists for each column of the final df
brand_id_list = []
brand_name_list = []
num_tot_posts_list = []
num_en_posts_list = []
num_it_posts_list = []
num_mturk_list = []
num_tribe_list = []
model_decision_empty_list = []

In [4]:
from pathlib import Path

# Loop over all the file ids provided
for brand_id in range(8009, 19151):
    
    brand_id_str = str(brand_id)
    
    # Check existence of file
    try_file = Path("../../Data/CSE_20180215/" + brand_id_str + "_data.json")
    
    if try_file.is_file():
        # file exists
        
        # Open json files
        with codecs.open('../../Data/CSE_20180215/' + brand_id_str + '_data.json', 'r', 'utf-8') as f_data:
            tweets_dict_list = json.load(f_data, encoding='utf-8')
        with codecs.open('../../Data/CSE_20180215/' + brand_id_str + '_metadata.json') as f_metadata:
            metadata_dict = json.load(f_metadata, encoding='utf-8')
        
        # Import as dataframe
        df = pd.DataFrame.from_dict(tweets_dict_list)
        
        # Fill the lists with the corresponding cell value
        brand_id_list.append(metadata_dict['brand_id'])
        brand_name_list.append(metadata_dict['brand_name'])
        num_tot_posts_list.append(df.shape[0])
        num_en_posts_list.append(df[df.lang == 'en'].shape[0])
        num_it_posts_list.append(df[df.lang == 'it'].shape[0])
        num_mturk_list.append(df[df.mturker == 1].shape[0])
        num_tribe_list.append(df[df.mturker != 1].shape[0])
        # Counter has one key if there is only 'None', otherwise has more
        n = len(Counter(df.model_decision.values))
        if n>1:
            model_decision_empty_list.append(False)
        else:
            model_decision_empty_list.append(True)

In [5]:
# Assign final values of the lists to df_res
df_res['brand_id'] = np.array(brand_id_list)
df_res['brand_name'] = np.array(brand_name_list)
df_res['num_tot_posts'] = np.array(num_tot_posts_list)
df_res['num_en_posts'] = np.array(num_en_posts_list)
df_res['num_it_posts'] = np.array(num_it_posts_list)
df_res['num_mturk'] = np.array(num_mturk_list)
df_res['num_tribe'] = np.array(num_tribe_list)
df_res['model_decision_empty'] = np.array(model_decision_empty_list)

In [6]:
df_res.head(10)

,brand_id,brand_name,num_tot_posts,num_en_posts,num_it_posts,num_mturk,num_tribe,model_decision_empty
0,8009,Caudalie,1188,706,57,468,720,True
1,8033,Mary Kay,3917,2094,14,3342,575,True
2,8499,Too Faced,1299,1005,48,368,931,True
3,9305,Avon,4107,2814,250,1690,2417,True
4,9337,Kérastase,821,505,47,133,688,True
5,9932,Givenchy,355,262,15,259,96,True
6,9996,Lancôme,1418,956,37,303,1115,True
7,10118,NARS,1391,910,29,300,1091,True
8,10195,Neutrogena,716,419,1,231,485,True
9,10918,Anastasia Beverly Hills,1245,1040,20,58,1187,True


In [7]:
# Output df to Excel file
writer = pd.ExcelWriter('new_brands_info.xlsx')
df_res.to_excel(writer,'Sheet1')
writer.save()